### Introduction: Taxi Fare Prediction###


Welcome to the Taxi Fare Kaggle challenge. In this contest, the aim is to predict the fare of a taxi ride given the starting time, the starting and ending latitude / longitude, and the number of passengers. 

In this kernel I am going to demonstrate the basic steps to tackle with a real life machine learning challenge. 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

We would keep the first 2m records for faster processing

In [ ]:
train_data = pd.read_csv("/kaggle/input/new-york-city-taxi-fare-prediction/train.csv", nrows = 2000000)
test_data = pd.read_csv("/kaggle/input/new-york-city-taxi-fare-prediction/test.csv", nrows = 20000)

In [ ]:
print(train_data.shape)
print(test_data.shape)

train_data["dataset"] = "train"
test_data["dataset"] = "test"

In [ ]:
X = pd.concat((train_data, test_data))
X.describe()

In [ ]:
# X = train_data.drop(("fare_amount"), axis = 1)
# y = train_data[["fare_amount"]]
# X = train_data

In [ ]:
print('Train columns with null values:\n', X.isnull().sum())
print("-"*10)

In [ ]:
f, ax = plt.subplots(figsize=(7, 7))
_ = sns.distplot(X[["fare_amount"]], fit=norm)

## Exclude Outlier

The starting fare is $2.5. So we exclude fares with extreme values. 

source : https://www1.nyc.gov/site/tlc/passengers/taxi-fare.page#:~:text=%242.50%20initial%20charge.,Dutchess%2C%20Orange%20or%20Putnam%20Counties.

In [ ]:
X = X[X['fare_amount'].between(left = 2.5, right = 100)& (X["dataset"]== "train") | (X["dataset"]=="test")]

In [ ]:
f, ax = plt.subplots(figsize=(10, 10))
k = ["pickup_longitude", "pickup_latitude","dropoff_longitude","dropoff_longitude"]
for i, field in enumerate(k, 1):
    plt.subplot(2,2,i)
    _ = sns.kdeplot(X[field])
    plt.xlabel([field])


In [ ]:
for col in ['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude']:
    print(f'{col.capitalize():20}: 2.5% = {round(np.percentile(X[col], 2.5), 2):5} \t 97.5% = {round(np.percentile(X[col], 97.5), 2)}')

Exclude the outlier

In [ ]:
# valid location of New York Manhattan -73.844311	40.721319
# X = X[(X["pickup_longitude"] != 0) & (X["dropoff_latitude"] != 0)& (X["pickup_latitude"] != 0)& (X["dropoff_longitude"] != 0)]
X = X[(X["pickup_longitude"] < -70) & (X["pickup_longitude"] > -76) & (X["pickup_latitude"] < 43) & (X["pickup_latitude"] > 37) &
     (X["dropoff_longitude"] < -70) & (X["dropoff_longitude"] > -76) & (X["dropoff_latitude"] < 43) & (X["dropoff_latitude"] > 37)]

In [ ]:
fig, axes = plt.subplots(1, 2, figsize = (20, 8), sharex=True, sharey=True)
axes = axes.flatten()

# Plot Longitude (x) and Latitude (y)
sns.regplot('pickup_longitude', 'pickup_latitude', fit_reg = False, 
            data  = X.sample(10000, random_state = 100), ax = axes[0]);
sns.regplot('dropoff_longitude', 'dropoff_latitude', fit_reg = False, 
            data  = X.sample(10000, random_state = 100), ax = axes[1]);
axes[0].set_title('Pickup Locations')
axes[1].set_title('Dropoff Locations');

Plot the folium map and we can see some coordinates falling in the sea. They should be wrong inputs but let's disregard them at this moment.

In [ ]:
# sas install folium
import folium
from folium.plugins import MarkerCluster
from folium import plugins
from folium.plugins import HeatMap

df_txn_display_pickup = X[:50000]

m=folium.Map([ 40.72,-74.00],zoom_start=10)
HeatMap(df_txn_display_pickup[['dropoff_latitude','dropoff_longitude']].dropna(),radius=8,gradient={0.2:'blue',0.4:'purple',0.6:'orange',1.0:'red'}).add_to(m)
m


In [ ]:
n=folium.Map([ 40.72,-74.00],zoom_start=10)
HeatMap(df_txn_display_pickup[['pickup_latitude','pickup_longitude']].dropna(),radius=8,gradient={0.2:'blue',0.4:'purple',0.6:'orange',1.0:'red'}).add_to(n)
n

Binning the fare 


In [ ]:
X

In [ ]:
X.groupby("dataset")["dataset"].agg("count")

In [ ]:
# Bin the fare and convert to string
X['fare-bin'] = pd.cut(X['fare_amount'], bins = list(range(0, 50, 5))).astype(str)

# Uppermost bin
X.loc[X['fare-bin'] == 'nan', 'fare-bin'] = '[45+]'

# Adjust bin so the sorting is correct
X.loc[X['fare-bin'] == '(5.0, 10.0]', 'fare-bin'] = '(05, 10]'

# Bar plot of value counts
X['fare-bin'].value_counts().sort_index().plot.bar(color = 'b', edgecolor = 'k');
plt.title('Fare Binned');

In [ ]:
def ecdf(x):
    """Empirical cumulative distribution function of a variable"""
    # Sort in ascending order
    x = np.sort(x)
    n = len(x)
    
    # Go from 1/n to 1
    y = np.arange(1, n + 1, 1) / n
    
    return x, y

In [ ]:
xs, ys = ecdf(X['fare_amount'])
plt.figure(figsize = (8, 6))
plt.plot(xs, ys, '.')
plt.ylabel('Percentile')
plt.title('ECDF of Fare Amount')
plt.xlabel('Fare Amount ($)')

In [ ]:
X['passenger_count'].value_counts().plot.bar(color = 'b', edgecolor = 'k');
plt.title('Passenger Counts'); plt.xlabel('Number of Passengers'); plt.ylabel('Count');



In [ ]:
X = X[X['passenger_count'].between(left = 1, right = 6)]

In [ ]:
# Create a color mapping based on fare bins
palette = sns.color_palette('Paired', 10)
color_mapping = {fare_bin: palette[i] for i, fare_bin in enumerate(X['fare-bin'].unique())}
color_mapping

X['color'] = X['fare-bin'].map(color_mapping)
plot_data = X.sample(1_000_000, random_state = 100)

In [ ]:
X

In [ ]:
BB_zoom = (-74.1, -73.7, 40.6, 40.85)
nyc_map_zoom = plt.imread('https://github.com/WillKoehrsen/Machine-Learning-Projects/blob/master/images/nyc_-74.1_-73.7_40.6_40.85.PNG?raw=true')

In [ ]:

fig, axs = plt.subplots(1, 1, figsize=(20, 18))

# Plot the pickups
for b, df in plot_data.groupby('fare-bin'):
    # Set the zorder to 1 to plot on top of map
    axs.scatter(df.pickup_longitude, df.pickup_latitude, zorder=1, alpha=0.2, c=df.color, s=30, label = f'{b}')
    axs.set_title('Pickup locations', size = 22)
    axs.axis('off')

# Legend
leg = axs.legend(fontsize = 14, markerscale = 3)

# Adjust alpha of legend markers
for lh in leg.legendHandles: 
    lh.set_alpha(1)

leg.set_title('Fare Bin', prop = {'size': 28})

# Show map in background (zorder = 0)
axs.imshow(nyc_map_zoom, zorder=0, extent=BB_zoom);

In [ ]:
# Absolute difference in latitude and longitude
X['abs_lat_diff'] = (X['dropoff_latitude'] - X['pickup_latitude']).abs()
X['abs_lon_diff'] = (X['dropoff_longitude'] - X['pickup_longitude']).abs()

In [ ]:
sns.lmplot('abs_lat_diff', 'abs_lon_diff', hue = 'fare-bin', size = 8, palette=palette,
           fit_reg = False, data = X.sample(10000, random_state=100));
plt.title('Absolute latitude difference vs Absolute longitude difference');

In [ ]:
sns.lmplot('abs_lat_diff', 'abs_lon_diff', hue = 'fare-bin', size = 8, palette=palette,
           fit_reg = False, data = X.sample(10000, random_state=100));
plt.title('Absolute latitude difference vs Absolute longitude difference');

plt.xlim((-0.01, .25)); plt.ylim((-0.01, .25))
plt.title('Absolute latitude difference vs Absolute longitude difference');

In [ ]:
no_diff = X[(X['abs_lat_diff'] == 0) & (X['abs_lon_diff'] == 0)]
no_diff.shape

There is 20788 records with no change in distance

In [ ]:
X

In [ ]:
import datetime as dt
import time

# X["fulldatetime"] = X["pickup_datetime"]
# X["fulldatetime"] = [a.strip(" UTC") for a in X["fulldatetime"]]
# X["date"] = [dt.dt.strptime(b, '%Y-%m-%d %H:%M:%S').date() for b in X["fulldatetime"]]
# X["time"] = [datetime.datetime.strptime(b, '%Y-%m-%d %H:%M:%S').time() for b in X["fulldatetime"]]

def add_datetime_info(dataset):
    #Convert to datetime format
    dataset['pickup_datetime'] = pd.to_datetime(dataset['pickup_datetime'],format="%Y-%m-%d %H:%M:%S UTC")
    
    dataset['hour'] = dataset.pickup_datetime.dt.hour
    dataset['day'] = dataset.pickup_datetime.dt.day
    dataset['month'] = dataset.pickup_datetime.dt.month
    dataset['weekday'] = dataset.pickup_datetime.dt.weekday
    dataset['year'] = dataset.pickup_datetime.dt.year
    
    return dataset

add_datetime_info(X)

# Convert date to ordinal date 
# example :
# >>> date2
# datetime.datetime(2012, 2, 2, 0, 0)
# >>> tstamp = time.mktime(date2.timetuple())
# >>> tstamp
# 1328121000.0

X["fulldatetimetuple"] = [time.mktime(a.timetuple()) for a in X["pickup_datetime"]]


# Euclidean Distance 
X["straightdist"] = ((X["pickup_longitude"] - X["dropoff_longitude"])**2 + (X["pickup_latitude"] - X["dropoff_latitude"])**2)**0.5

# Manhattan Distance
X["manhattan"] = X['abs_lat_diff'] + X['abs_lon_diff'] 


# Haversine Distance 
from haversine import haversine, Unit

X['pickup'] = list(zip(X.pickup_latitude, X.pickup_longitude))
X['dropoff'] = list(zip(X.dropoff_latitude, X.dropoff_longitude))

def calculate_haversine(row):
    dis = haversine(row['pickup'], row['dropoff'])
    return dis

X["haverdist"] = X.apply(calculate_haversine, axis=1)

In [ ]:
  
 # Calculate distribution by each fare bin
plt.figure(figsize = (12, 6))
for f, grouped in X.groupby('fare-bin'):
    sns.kdeplot(grouped['manhattan'], label = f'{f}', color = list(grouped['color'])[0]);

plt.xlabel('degrees'); plt.ylabel('density')
plt.title('Manhattan Distance by Fare Amount');

In [ ]:
  
 # Calculate distribution by each fare bin
plt.figure(figsize = (12, 6))
for f, grouped in X.groupby('fare-bin'):
    sns.kdeplot(grouped['haverdist'], label = f'{f}', color = list(grouped['color'])[0]);

plt.xlabel('degrees'); plt.ylabel('density')
plt.title('Haverdist Distance by Fare Amount');

In [ ]:
# Calculate distribution by each fare bin
plt.figure(figsize = (12, 6))
for f, grouped in X.groupby('fare-bin'):
    sns.kdeplot(grouped['straightdist'], label = f'{f}', color = list(grouped['color'])[0]);

plt.xlabel('degrees'); plt.ylabel('density')
plt.title('Euclidean Distance by Fare Amount');

In [ ]:
X.groupby('fare-bin')['straightdist'].agg(['mean', 'count'])

In [ ]:
X.groupby('fare-bin')['straightdist'].mean().plot.bar(color = 'b');
plt.title('Average Euclidean Distance by Fare Bin');

In [ ]:
plt.figure(figsize = (10, 6))

for p, grouped in X.groupby('passenger_count'):
    sns.kdeplot(grouped['fare_amount'], label = f'{p} passengers', color = list(grouped['color'])[0]);
    
plt.xlabel('Fare Amount'); plt.ylabel('Density')
plt.title('Distribution of Fare Amount by Number of Passengers');


In [ ]:
X.groupby('passenger_count')['fare_amount'].agg(['mean', 'count'])

In [ ]:
X.groupby('passenger_count')['fare_amount'].mean().plot.bar(color = 'b');
plt.title('Average Fare by Passenger Count');

In [ ]:
# f, ax = plt.subplots(figsize=(7, 7))
# _ = sns.pairplot(X.sample(1000, random_state = 100))

In [ ]:
corrs = X.corr()
corrs['fare_amount'].plot.bar(color = 'b');
plt.title('Correlation with Fare Amount');


In [ ]:
X.sample(5)

In [ ]:
f, ax = plt.subplots(figsize=(7, 7))
sns.lineplot(x="year", y="fare_amount",data=X)

#### Year : fare amount is higher generally in post 2013 compared to pre 2013 


#### Weekday / day / month : no trend observed

In [ ]:
f, ax = plt.subplots(figsize=(10, 10))
k = ["year", "weekday","month","day"]
for i, field in enumerate(k, 1):
    plt.subplot(2,2,i)
    _ = sns.boxplot(y = X["fare_amount"], x = X[field], palette="Blues")
    plt.xlabel([field])


In [ ]:
f, ax = plt.subplots(figsize=(10, 10))
k = ["straightdist", "hour"]
for i, field in enumerate(k, 1):
    plt.subplot(2,1,i)
    _ = sns.scatterplot(y = X["fare_amount"], x = X[field], palette="Blues")
    plt.xlabel([field])
    
# sns.scatterplot(x="straightdist", y="fare_amount",data=X)

In [ ]:
# f, ax = plt.subplots(figsize=(10, 10))
e = sns.FacetGrid(X, col =  "dataset")
e.map(sns.scatterplot, "haverdist", "fare_amount") 

#no fare amount in test dataset

In [ ]:
f = sns.FacetGrid(X, col =  "dataset")
f.map(sns.distplot, "haverdist")

In [ ]:
# Only 412 records with haverdist > 30km out of 2m records
X[X["haverdist"]>100]

X = X[X["haverdist"]<100]

# delete negative fare for training data
X = X[ ((X["fare_amount"]>0) & (X["dataset"]== "train")) | (X["dataset"]=="test")]

In [ ]:
# X[(X["haverdist"]<0.2) & (X["fare_amount"]>10)] 

# X = X.drop(X[(X["haverdist"]<0.2) & (X["fare_amount"]>10)].index)

In [ ]:
plt.figure(figsize=(10,6))

k = {"dropoff" : ["dropoff_longitude",  "dropoff_latitude"],
     "pickup" : ["pickup_longitude",  "pickup_latitude"]}
for i, field in enumerate(k, 1):
    plt.subplot(2,1,i)
    _ = sns.scatterplot(y = X[k[field][1]], x = X[k[field][0]], palette="Blues")
    plt.xlabel(k[field][0])
    plt.ylabel(k[field][1])

In [ ]:
# sas install folium
import folium
from folium.plugins import MarkerCluster
from folium import plugins
from folium.plugins import HeatMap

df_txn_display_pickup = X[:50000]

m=folium.Map([ 40.72,-74.00],zoom_start=10)
HeatMap(df_txn_display_pickup[['dropoff_latitude','dropoff_longitude']].dropna(),radius=8,gradient={0.2:'blue',0.4:'purple',0.6:'orange',1.0:'red'}).add_to(m)
m


In [ ]:
n=folium.Map([ 40.72,-74.00],zoom_start=10)
HeatMap(df_txn_display_pickup[['pickup_latitude','pickup_longitude']].dropna(),radius=8,gradient={0.2:'blue',0.4:'purple',0.6:'orange',1.0:'red'}).add_to(n)
n

In [ ]:
X.describe()

In [ ]:
X[X["straightdist"]==None]

In [ ]:
X = X.drop(X[X["straightdist"].isna()].index)  #drop straightdist = NA
X

In [ ]:
X[X.isna().any(axis=1)]  #display all NA 

In [ ]:
X.isna().sum()

In [ ]:
#histogram and normal probability plot\
from scipy.stats import norm
import scipy.stats as stats

sns.distplot(X['fare_amount'], fit=norm);
fig = plt.figure()
res = stats.probplot(X['fare_amount'],  plot=plt)


In [ ]:
X["logfare"] = np.log1p(X['fare_amount'])

sns.distplot(X["logfare"] , fit=norm);
fig = plt.figure()
res = stats.probplot(X['logfare'],  plot=plt)



In [ ]:
from scipy.special import boxcox1p
X.skew(axis = 0, skipna = True) 
lam = 0.15

X["skewhaverdist"] = boxcox1p(X["haverdist"], lam)


In [ ]:
import re

def extract_dateinfo(df, date_col, drop=True, time=False, 
                     start_ref = pd.datetime(1900, 1, 1),
                     extra_attr = False):
    """
    Extract Date (and time) Information from a DataFrame
    Adapted from: https://github.com/fastai/fastai/blob/master/fastai/structured.py
    """
    df = df.copy()
    
    # Extract the field
    fld = df[date_col]
    
    # Check the time
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    # Convert to datetime if not already
    if not np.issubdtype(fld_dtype, np.datetime64):
        df[date_col] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    

    # Prefix for new columns
    pre = re.sub('[Dd]ate', '', date_col)
    pre = re.sub('[Tt]ime', '', pre)
    
    # Basic attributes
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear', 'Days_in_month', 'is_leap_year']
    
    # Additional attributes
    if extra_attr:
        attr = attr + ['Is_month_end', 'Is_month_start', 'Is_quarter_end', 
                       'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    
    # If time is specified, extract time information
    if time: 
        attr = attr + ['Hour', 'Minute', 'Second']
        
    # Iterate through each attribute
    for n in attr: 
        df[pre + n] = getattr(fld.dt, n.lower())
        
    # Calculate days in year
    df[pre + 'Days_in_year'] = df[pre + 'is_leap_year'] + 365
        
    if time:
        # Add fractional time of day (0 - 1) units of day
        df[pre + 'frac_day'] = ((df[pre + 'Hour']) + (df[pre + 'Minute'] / 60) + (df[pre + 'Second'] / 60 / 60)) / 24
        
        # Add fractional time of week (0 - 1) units of week
        df[pre + 'frac_week'] = (df[pre + 'Dayofweek'] + df[pre + 'frac_day']) / 7
    
        # Add fractional time of month (0 - 1) units of month
        df[pre + 'frac_month'] = (df[pre + 'Day'] + (df[pre + 'frac_day'])) / (df[pre + 'Days_in_month'] +  1)
        
        # Add fractional time of year (0 - 1) units of year
        df[pre + 'frac_year'] = (df[pre + 'Dayofyear'] + df[pre + 'frac_day']) / (df[pre + 'Days_in_year'] + 1)
        
    # Add seconds since start of reference
    df[pre + 'Elapsed'] = (fld - start_ref).dt.total_seconds()
    
    if drop: 
        df = df.drop(date_col, axis=1)
        
    return df


In [ ]:
X = extract_dateinfo(X, 'pickup_datetime', drop = False, 
                         time = True, start_ref = X['pickup_datetime'].min())
X.head()

In [ ]:
plt.figure(figsize = (12, 12))
corrs["fare_amount"].sort_values(ascending=False).plot.bar(color = "red")

In [ ]:
corrs = X.corr()

plt.figure(figsize = (12, 12))
corrs["fare_amount"].sort_values(ascending=False).plot.bar(color = "red")

In [ ]:
X[X.isna().any(axis=1)]  #display all NA 

In [ ]:
# test_y = X[X["dataset"] == "test"]
# test_y = test_y[["fare_amount"]].values

# filter_test = X["dataset"] == "test"
# filter_train = X["dataset"] == "train"

# train_y  = X[filter_train][["fare_amount"]].values
# train_X  = X[filter_train][["haverdist"]].values
# test_X  = X[filter_test][["haverdist"]].values

# features = ['year', 'hour', 'haverdist', 'passenger_count']
# features = ['year', 'hour', 'skewhaverdist', 'passenger_count']
# features_y = ['logfare']
# features = [ 'haverdist']

# train_y  = X[filter_train][features_y].values
# train_X  = X[filter_train][features].values
# test_X  = X[filter_test][features].values

# train_X = X[filter_train].drop(features_todrop, axis = 1).values
# test_X = X[filter_test].drop(features_todrop, axis = 1).values
# train_X = X[filter_train].drop(["key","fare_amount","pickup_datetime","dataset","pickup","dropoff"], axis = 1).values
# test_X = X[filter_test].drop(["key","fare_amount","pickup_datetime","dataset","pickup","dropoff"], axis = 1).values
# filtering data 
# data.where(filter, inplace = True) 

In [ ]:
from sklearn.model_selection import train_test_split
X_tr, X_va, y_tr, y_va = train_test_split(X[X["dataset"]=="train"], np.array(X[X["dataset"]=="train"]['fare_amount']), test_size=0.25, random_state  = 100)

In [ ]:
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore', category = RuntimeWarning)

def metrics(train_pred, valid_pred, y_train, y_valid):
    """Calculate metrics:
       Root mean squared error and mean absolute percentage error"""
    
    # Root mean squared error
    train_rmse = np.sqrt(mean_squared_error(y_train, train_pred))
    valid_rmse = np.sqrt(mean_squared_error(y_valid, valid_pred))
    
    # Calculate absolute percentage error
    train_ape = abs((y_train - train_pred) / y_train)
    valid_ape = abs((y_valid - valid_pred) / y_valid)
    
    # Account for y values of 0
    train_ape[train_ape == np.inf] = 0
    train_ape[train_ape == -np.inf] = 0
    valid_ape[valid_ape == np.inf] = 0
    valid_ape[valid_ape == -np.inf] = 0
    
    train_mape = 100 * np.mean(train_ape)
    valid_mape = 100 * np.mean(valid_ape)
    
    return train_rmse, valid_rmse, train_mape, valid_mape

def evaluate(model, features, X_train, X_valid, y_train, y_valid):
    """Mean absolute percentage error"""
    
    # Make predictions
    train_pred = model.predict(X_train[features])
    valid_pred = model.predict(X_valid[features])
    
    # Get metrics
    train_rmse, valid_rmse, train_mape, valid_mape = metrics(train_pred, valid_pred,
                                                             y_train, y_valid)
    
    print(f'Training:   rmse = {round(train_rmse, 2)} \t mape = {round(train_mape, 2)}')
    print(f'Validation: rmse = {round(valid_rmse, 2)} \t mape = {round(valid_mape, 2)}')

In [ ]:
y_tr

In [ ]:
plt.figure(figsize = (12, 12))
corrs["fare_amount"].sort_values(ascending=False)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score


# y_tr is ['fare_amount']
# features = ['year', 'hour', 'skewhaverdist', 'passenger_count']
features = ['abs_lon_diff', 'abs_lat_diff','haverdist', 'passenger_count']
model = LinearRegression()
model.fit(X_tr[features], y_tr)
py = model.predict(X_tr[features])
py_val = model.predict(X_va[features])
rmse = np.sqrt(mean_squared_error(y_tr, py))
rmse2 = np.sqrt(mean_squared_error(y_va, py_val))
print("rmse for training:", rmse)
print("rmse for validation:", rmse2)

# change y_tr as y_tr2 = ['logfare']
y_tr2  = np.array(X_tr['logfare'])
y_va2  = np.array(X_va['logfare'])
model.fit(X_tr[features], y_tr2)
py2 = model.predict(X_tr[features])
py2_val = model.predict(X_va[features])
rmse__2 = np.sqrt(mean_squared_error(y_tr2, py2))
rmse__2_val = np.sqrt(mean_squared_error(y_va2, py2_val))
print("rmse for training:", rmse__2)
print("rmse for validation:", rmse__2_val)


# def cv_rmse(model, X, y):
#     rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv = 5).mean())
#     return print(rmse)
# cv_rmse(LinearRegression(), train_X, train_y)
# model.score(train_X, train_y)

In [ ]:
#Use Bill Method 
# from sklearn.model_selection import train_test_split
# X_tr, X_va, y_tr, y_va = train_test_split(X, np.array(X['fare_amount']), test_size=0.25)

lr = LinearRegression()
features = ['abs_lon_diff', 'abs_lat_diff','haverdist', 'passenger_count']


lr.fit(X_tr[features], y_tr)

print('Intercept', round(lr.intercept_, 4))
print('abs_lat_diff coef: ', round(lr.coef_[0], 4), 
      '\tabs_lon_diff coef:', round(lr.coef_[1], 4),
      '\thaverdist coef:', round(lr.coef_[2], 4),
      '\tpassenger_count coef:', round(lr.coef_[3], 4)
     )

evaluate(lr, features, X_tr, X_va, y_tr, y_va)

In [ ]:
features = ['abs_lat_diff', 'abs_lon_diff', 'haverdist']


lr.fit(X_tr[features], y_tr)

print('Intercept', round(lr.intercept_, 4))
print('abs_lat_diff coef: ', round(lr.coef_[0], 4), 
      '\tabs_lon_diff coef:', round(lr.coef_[1], 4),
      '\thaverdist coef:', round(lr.coef_[2], 4)
     )

evaluate(lr, features, X_tr, X_va, y_tr, y_va)

### Use Random Forest : RMSE is better @3.35, but we found overfitting to validation dataset. We use pruning to mitigate 

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Create the random forest
random_forest = RandomForestRegressor(n_estimators = 20, max_depth = 20, 
                                      max_features = None, oob_score = True, 
                                      bootstrap = True, verbose = 1, n_jobs = -1)

# Train on data
random_forest.fit(X_tr[['haverdist', 'abs_lat_diff', 'abs_lon_diff', 'passenger_count']], y_tr)
evaluate(random_forest, ['haverdist', 'abs_lat_diff', 'abs_lon_diff', 'passenger_count'],
         X_tr, X_va, y_tr, y_va)

In [ ]:
import pandas as pd
feature_importances = pd.DataFrame({'feature': ['haverdist', 'abs_lat_diff', 'abs_lon_diff', 'passenger_count'],
                                        'importance': random_forest.feature_importances_}).\
                           sort_values('importance', ascending = False).set_index('feature')
    

In [ ]:
feature_importances

In [ ]:
feature_importances.plot.bar(color = 'b', edgecolor = 'k', linewidth = 2);
plt.title('Feature Importances');

In [ ]:
X_tr.columns

In [ ]:
random_forest.fit(X_tr[['haverdist', 'abs_lat_diff', 'abs_lon_diff', 'passenger_count','pickup_Elapsed','manhattan','pickup_longitude','pickup_latitude', 'dropoff_longitude','dropoff_latitude']], y_tr)
evaluate(random_forest, ['haverdist', 'abs_lat_diff', 'abs_lon_diff', 'passenger_count','pickup_Elapsed','manhattan','pickup_longitude','pickup_latitude', 'dropoff_longitude','dropoff_latitude'],
         X_tr, X_va, y_tr, y_va)

Pruning : cut max_depth from 20 to 10; however, result is not satisfactory

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Create the random forest
random_forest = RandomForestRegressor(n_estimators = 20, max_depth = 10, 
                                      max_features = None, oob_score = True, 
                                      bootstrap = True, verbose = 1, n_jobs = -1)

# Train on data
random_forest.fit(X_tr[['haverdist', 'abs_lat_diff', 'abs_lon_diff', 'passenger_count','pickup_Elapsed','manhattan','pickup_longitude','pickup_latitude', 'dropoff_longitude','dropoff_latitude']], y_tr)
evaluate(random_forest, ['haverdist', 'abs_lat_diff', 'abs_lon_diff', 'passenger_count','pickup_Elapsed','manhattan','pickup_longitude','pickup_latitude', 'dropoff_longitude','dropoff_latitude'],
         X_tr, X_va, y_tr, y_va)

### Linear Regression + Standard Scalar

In [ ]:
# define some handy analysis support function
from sklearn.metrics import mean_squared_error, explained_variance_score

def plot_prediction_analysis(y, y_pred, figsize=(10,4), title=''):
    fig, axs = plt.subplots(1, 2, figsize=figsize)
    axs[0].scatter(y, y_pred)
    mn = min(np.min(y), np.min(y_pred))
    mx = max(np.max(y), np.max(y_pred))
    axs[0].plot([mn, mx], [mn, mx], c='red')
    axs[0].set_xlabel('$y$')
    axs[0].set_ylabel('$\hat{y}$')
    rmse = np.sqrt(mean_squared_error(y, y_pred))
    evs = explained_variance_score(y, y_pred)
    axs[0].set_title('rmse = {:.2f}, evs = {:.2f}'.format(rmse, evs))
    
    axs[1].hist(y-y_pred, bins=50)
    avg = np.mean(y-y_pred)
    std = np.std(y-y_pred)
    axs[1].set_xlabel('$y - \hat{y}$')
    axs[1].set_title('Histrogram prediction error, $\mu$ = {:.2f}, $\sigma$ = {:.2f}'.format(avg, std))
    
    if title!='':
        fig.suptitle(title)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

model_lin = Pipeline((
        ("standard_scaler", StandardScaler()),
        ("lin_reg", LinearRegression()),
    ))


features = ['haverdist', 'abs_lat_diff', 'abs_lon_diff', 'passenger_count']
model_lin.fit(X_tr[features], y_tr)


y_train_pred = model_lin.predict(X_tr[features])
plot_prediction_analysis(y_tr, y_train_pred, title='Linear Model - Trainingset')

y_test_pred = model_lin.predict(X_va[features])
plot_prediction_analysis(y_va, y_test_pred, title='Linear Model - Testset')

In [ ]:
def plot_rmse_analysis(model, X, y, N=400, test_size=0.25, figsize=(10,4), title=''):
    rmse_train, rmse_test = [], []
    for i in range(N):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)

        model.fit(X_train, y_train)
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)

        rmse_train.append(np.sqrt(mean_squared_error(y_train, y_train_pred)))
        rmse_test.append(np.sqrt(mean_squared_error(y_test, y_test_pred)))

    g = sns.jointplot(np.array(rmse_train), np.array(rmse_test), kind='scatter', stat_func=None, size=5)
    g.set_axis_labels("RMSE training ($\mu$={:.2f})".format(np.mean(rmse_train)), 
                      "RMSE test ($\mu$={:.2f})".format(np.mean(rmse_test)))
    plt.subplots_adjust(top=0.9)
    g.fig.suptitle('{} (N={}, test_size={:0.2f})'.format(title, N, test_size))

plot_rmse_analysis(model_lin, X_tr[features], y_tr, title='Linear model')  
    

In [ ]:
X_tr.columns

Adding time features 

In [ ]:
time_features = ['pickup_Year', 'pickup_Month',
       'pickup_Week', 'pickup_Day', 'pickup_Dayofweek', 'pickup_Dayofyear',
       'pickup_Days_in_month', 'pickup_is_leap_year', 'pickup_Hour',
       'pickup_Minute', 'pickup_Second', 'pickup_Days_in_year',
       'pickup_frac_day', 'pickup_frac_week', 'pickup_frac_month',
       'pickup_frac_year', 'pickup_Elapsed']

features2 = ['abs_lat_diff', 'abs_lon_diff', 'haverdist', 'passenger_count',
            'pickup_latitude', 'pickup_longitude', 
            'dropoff_latitude', 'dropoff_longitude','manhattan'] + time_features

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Create the random forest
random_forest = RandomForestRegressor(n_estimators = 20, max_depth = 20, 
                                      max_features = None, oob_score = True, 
                                      bootstrap = True, verbose = 1, n_jobs = -1)

# Train on data
random_forest.fit(X_tr[features2], y_tr)
evaluate(random_forest, features2,
         X_tr, X_va, y_tr, y_va)

In [ ]:
col = X_tr[features2].columns.tolist()
col2 = np.array(col)
col2.shape

import pandas as pd
feature_importances = pd.DataFrame({'feature': col2,
                                        'importance': random_forest.feature_importances_}).\
                           sort_values('importance', ascending = False).set_index('feature')

feature_importances.plot.bar(color = 'b', edgecolor = 'k', linewidth = 2);
plt.title('Feature Importances');


Fineturning the parameters to mitigate overfitting

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Hyperparameter grid
param_grid = {
    'n_estimators': np.linspace(10, 100).astype(int),
    'max_depth': [None] + list(np.linspace(5, 30).astype(int)),
    'max_features': ['auto', 'sqrt', None] + list(np.arange(0.5, 1, 0.1)),
    'max_leaf_nodes': [None] + list(np.linspace(10, 50, 500).astype(int)),
    'min_samples_split': [2, 5, 10],
    'bootstrap': [True, False]
}

# Estimator for use in random search
estimator = RandomForestRegressor(random_state = 100)

# Create the random search model, change "interation"
rs = RandomizedSearchCV(estimator, param_grid, n_jobs = -1, 
                        scoring = 'neg_mean_absolute_error', cv = 3, 
                        n_iter = 5, verbose = 1, random_state=100)


# Train on data
rs.fit(X_tr[features2], y_tr)
bestmodel = rs.best_estimator_
bestmodelscore = rs.best_score_

In [ ]:
bestmodel

In [ ]:
y_train_bestrf = bestmodel.predict(X_tr[features2])
rmse = np.sqrt(mean_squared_error(y_train_bestrf, y_tr))
print("train_rmse = ", rmse)

y_valid_bestrf  = bestmodel.predict(X_va[features2])
rmse = np.sqrt(mean_squared_error(y_valid_bestrf , y_va))
print("valid_rmse = ", rmse)

### Gradient Boosting ###

In [ ]:
import lightgbm as lgbm

# features = ['haverdist', 'abs_lat_diff', 'abs_lon_diff', 'passenger_count','pickup_Elapsed','manhattan','pickup_longitude','pickup_latitude', 'dropoff_longitude','dropoff_latitude']

params = {
        'boosting_type':'gbdt',
        'objective': 'regression',
        'nthread': -1,
        'verbose': 0,
        'num_leaves': 31,
        'learning_rate': 0.05,
        'max_depth': -1,
        'subsample': 0.8,
        'subsample_freq': 1,
        'colsample_bytree': 0.6,
        'reg_aplha': 1,
        'reg_lambda': 0.001,
        'metric': 'rmse',
        'min_split_gain': 0.5,
        'min_child_weight': 1,
        'min_child_samples': 10,
        'scale_pos_weight':1     
    }
train_set = lgbm.Dataset(X_tr[features2], y_tr)
valid_set = lgbm.Dataset(X_va[features2], y_va)

gb = lgbm.train(params, train_set = train_set, num_boost_round=300)

In [ ]:
y_train_gb = gb.predict(X_tr[features2])
rmse = np.sqrt(mean_squared_error(y_train_gb, y_tr))
print("train_rmse = ", rmse)

y_valid_gb = gb.predict(X_va[features2])
rmse = np.sqrt(mean_squared_error(y_valid_gb, y_va))
print("valid_rmse = ", rmse)

In [ ]:
import xgboost as xgb

# features = ['haverdist', 'abs_lat_diff', 'abs_lon_diff', 'passenger_count','pickup_Elapsed','manhattan','pickup_longitude','pickup_latitude', 'dropoff_longitude','dropoff_latitude']


dtrain_xg = xgb.DMatrix(X_tr[features2], label=y_tr)
dval_xg = xgb.DMatrix(X_va[features2])

params = {'max_depth':7,
          'eta':1,
          'silent':1,
          'objective':'reg:linear',
          'eval_metric':'rmse',
          'learning_rate':0.05
         }
num_rounds = 50
xb = xgb.train(params, dtrain_xg, num_rounds)


# y_train_xgb = xb.predict(dtrain)
# rmse = np.sqrt(mean_squared_error(y_train_xgb, y_tr))
# rmse

In [ ]:
y_train_xg = xb.predict(dtrain_xg)
rmse = np.sqrt(mean_squared_error(y_train_xg, y_tr))
print("train_rmse = ", rmse)


y_valid_xg = xb.predict(dval_xg)
rmse2 = np.sqrt(mean_squared_error(y_valid_xg, y_va))
print("valid_rmse = ", rmse2)

In [ ]:
xgb.plot_importance(xb)


### MODEL SUBMISSION ###

In [ ]:
# predict_y = model.predict(test_X)

# rerunning the model include the testing dataset 
# xb = xgb.train(params, dtrain, num_rounds)
predict_y = gb.predict(X[X["dataset"] == "test"][features2])

# predict_y = model_lin.predict(test_X)
# predict_y = np.expm1(predict_y)

In [ ]:
predict_y.shape

In [ ]:
# predict_y = predict_y[:,0]

In [ ]:
#Create submission file
submission = pd.DataFrame({'key': X[X["dataset"] == "test"]["key"],
    'fare_amount':predict_y.round(2)
})


submission.to_csv('taxi_fare_submission.csv',index=False)
submission.head()

## Special Thanks to 

https://www.kaggle.com/danpavlov/ny-taxi-fare-comprehensive-and-simple-analysis


https://www.kaggle.com/willkoehrsen/a-walkthrough-and-a-challenge